# Susceptibility Scores
A notebook for initial exploration.

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [2]:
import os
import sys
import random
from tqdm import tqdm

from transformers import GPTNeoXForCausalLM, AutoTokenizer
import torch
import numpy as np
import wandb

/home/kevin/mambaforge/envs/measurelm2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from measuring.estimate_probs import estimate_cmi
from preprocessing.datasets import CountryCapital, FriendEnemy

### Preamble

In [4]:
##################
### Parameters ###
##################

# Data parameters
SEED = 0
DATASET_NAME = "CountryCapital"
DATASET_KWARGS_IDENTIFIABLE = dict(
    max_contexts=15,
    max_entities=5,
    cap_per_type=True,
    raw_country_capitals_path="data/CountryCapital/real-fake-historical-fictional-famousfictional-country-capital.csv",
    ablate_out_relevant_contexts=True,
)
# DATASET_KWARGS_IDENTIFIABLE = dict(
#     max_contexts=450,
#     max_entities=90,
#     cap_per_type=True,
#     raw_country_capitals_path="data/CountryCapital/real-fake-historical-fictional-famousfictional-country-capital.csv",
#     ablate_out_relevant_contexts=True,
# )
# DATASET_NAME = "FriendEnemy"
# DATASET_KWARGS_IDENTIFIABLE = dict(
#     max_contexts=15,
#     max_entities=5,
#     cap_per_type=False,
#     raw_data_path="data/FriendEnemy/raw-friend-enemy.csv",
# )
# DATASET_KWARGS_IDENTIFIABLE = dict(
#     max_contexts=657,
#     max_entities=73,
#     cap_per_type=False,
#     raw_data_path="data/FriendEnemy/raw-friend-enemy.csv",
# )
LOG_DATASETS = True

# Model parameters
MODEL_ID = "EleutherAI/pythia-70m-deduped"
LOAD_IN_8BIT = False
# MODEL_ID = "EleutherAI/pythia-6.9b-deduped"
# LOAD_IN_8BIT = True
BATCH_SZ = 16

# Evaluation switches
COMPUTE_CMI = True
COMPUTE_KL = True
COMPUTE_GOOD_BAD = True
COMPUTE_GOOD_BAD_ABS = True
COMPUTE_GOOD_BAD_P_GOOD_ONLY = True

# wandb stuff
PROJECT_NAME = "context-vs-bias"
GROUP_NAME = None
TAGS = ["capitals"]
# TAGS = ["friend-enemy"]

In [5]:
# Set random seeds
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [6]:
# Paths
# Construct dataset and data ids
# dataset = getattr(sys.modules[__name__], DATASET_NAME)(**DATASET_KWARGS_IDENTIFIABLE)
data_id = f"{DATASET_NAME}"
data_id += (
    f"-mc{DATASET_KWARGS_IDENTIFIABLE['max_contexts']}"
    if "max_contexts" in DATASET_KWARGS_IDENTIFIABLE
    and DATASET_KWARGS_IDENTIFIABLE["max_contexts"] is not None
    else ""
)
data_id += (
    f"-me{DATASET_KWARGS_IDENTIFIABLE['max_entities']}"
    if "max_entities" in DATASET_KWARGS_IDENTIFIABLE
    and DATASET_KWARGS_IDENTIFIABLE["max_entities"] is not None
    else ""
)
data_id += (
    "-cappertype"
    if "cap_per_type" in DATASET_KWARGS_IDENTIFIABLE
    and DATASET_KWARGS_IDENTIFIABLE["cap_per_type"]
    else ""
)


data_dir = os.path.join("data", DATASET_NAME, data_id, f"{SEED}")
input_dir = os.path.join(data_dir, "inputs")
entities_path = os.path.join(input_dir, "entities.json")
contexts_path = os.path.join(input_dir, "contexts.json")
queries_path = os.path.join(input_dir, "queries.json")
val_data_path = os.path.join(input_dir, "val.csv")
DATASET_KWARGS_IDENTIFIABLE = {
    **DATASET_KWARGS_IDENTIFIABLE,
    **dict(
        entities_path=entities_path,
        contexts_path=contexts_path,
        queries_path=queries_path,
    ),
}

results_dir = os.path.join(data_dir, "results")
val_results_path = os.path.join(results_dir, "val.csv")

# Construct model id
model_id = f"{MODEL_ID}"
model_id += "-8bit" if LOAD_IN_8BIT else ""
model_dir = os.path.join(data_dir, "models", model_id)

print(f"Data dir: {data_dir}")
print(f"Model dir: {model_dir}")

Data dir: data/CountryCapital/CountryCapital-mc15-me5-cappertype/0
Model dir: data/CountryCapital/CountryCapital-mc15-me5-cappertype/0/models/EleutherAI/pythia-70m-deduped


In [7]:
os.makedirs(input_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
dataset = getattr(sys.modules[__name__], DATASET_NAME)(**DATASET_KWARGS_IDENTIFIABLE)

In [8]:
# GPU stuff
device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
# wandb stuff
os.environ["WANDB_NOTEBOOK_NAME"] = os.path.join(os.getcwd(), "main.ipynb")

params_to_log = {k: v for k, v in locals().items() if k.isupper()}

run = wandb.init(
    project=PROJECT_NAME,
    group=GROUP_NAME,
    config=params_to_log,
    tags=TAGS,
    mode="online",
)
print(dict(wandb.config))

wandb: Currently logged in as: kdu (ethz-rycolab). Use `wandb login --relogin` to force relogin


{'SEED': 0, 'DATASET_NAME': 'CountryCapital', 'DATASET_KWARGS_IDENTIFIABLE': {'max_contexts': 15, 'max_entities': 5, 'cap_per_type': True, 'raw_country_capitals_path': 'data/CountryCapital/real-fake-historical-fictional-famousfictional-country-capital.csv', 'ablate_out_relevant_contexts': True, 'entities_path': 'data/CountryCapital/CountryCapital-mc15-me5-cappertype/0/inputs/entities.json', 'contexts_path': 'data/CountryCapital/CountryCapital-mc15-me5-cappertype/0/inputs/contexts.json', 'queries_path': 'data/CountryCapital/CountryCapital-mc15-me5-cappertype/0/inputs/queries.json'}, 'LOG_DATASETS': True, 'MODEL_ID': 'EleutherAI/pythia-70m-deduped', 'LOAD_IN_8BIT': False, 'BATCH_SZ': 16, 'COMPUTE_CMI': True, 'COMPUTE_KL': True, 'COMPUTE_GOOD_BAD': True, 'COMPUTE_GOOD_BAD_ABS': True, 'COMPUTE_GOOD_BAD_P_GOOD_ONLY': True, 'PROJECT_NAME': 'context-vs-bias', 'GROUP_NAME': None, 'TAGS': ['capitals']}


### Load Data

In [10]:
val_df_contexts_per_qe = dataset.get_contexts_per_query_entity_df()

if LOG_DATASETS:
    print(f"Saving datasets to {input_dir}.")
    os.makedirs(input_dir, exist_ok=True)
    val_df_contexts_per_qe.to_csv(val_data_path)

val_df_contexts_per_qe.info()
val_df_contexts_per_qe.head()

Saving datasets to data/CountryCapital/CountryCapital-mc15-me5-cappertype/0/inputs.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   q_id        10 non-null     object
 1   query_form  10 non-null     object
 2   entity      10 non-null     object
 3   contexts    10 non-null     object
dtypes: object(4)
memory usage: 448.0+ bytes


,q_id,query_form,entity,contexts
0,capital_of,Q: What is the capital of {}?\nA:,"(Zimbabwe,)",[The capital of Part of Kingdom of Sicily is A...
1,capital_of,Q: What is the capital of {}?\nA:,"(The Sticklands,)",[The capital of Part of Kingdom of Sicily is A...
2,capital_of,Q: What is the capital of {}?\nA:,"(Jakana,)",[The capital of Part of Kingdom of Sicily is A...
3,capital_of,Q: What is the capital of {}?\nA:,"(Mordor,)",[The capital of Part of Kingdom of Sicily is A...
4,capital_of,Q: What is the capital of {}?\nA:,"(Baekje,)",[The capital of Part of Kingdom of Sicily is A...


### Preprocess Data

In [11]:
# Preprocess the data and convert it into inputs for the model (e.g. torch tensors)

In [12]:
# After loading/preprocessing your dataset, log it as an artifact to W&B
if LOG_DATASETS:
    print(f"Logging datasets to w&b run {wandb.run}.")
    artifact = wandb.Artifact(name=data_id, type="dataset")
    artifact.add_dir(local_path=data_dir)
    run.log_artifact(artifact)

wandb: Adding directory to artifact (./data/CountryCapital/CountryCapital-mc15-me5-cappertype/0)... Done. 0.0s


Logging datasets to w&b run <wandb.sdk.wandb_run.Run object at 0x7f89e480c2b0>.


### Score Model

In [13]:
try:
    model = GPTNeoXForCausalLM.from_pretrained(
        MODEL_ID, load_in_8bit=LOAD_IN_8BIT, device_map="auto"
    )
except:
    print(f"Failed to load model {MODEL_ID} in 8-bit. Attempting to load normally.")
    model = GPTNeoXForCausalLM.from_pretrained(
        MODEL_ID,
        load_in_8bit=False,
    ).to(device)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    padding_side="left",
)

Failed to load model EleutherAI/pythia-70m-deduped in 8-bit. Attempting to load normally.


RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
!nvidia-smi --query-gpu=memory.used --format=csv

memory.used [MiB]
879 MiB


In [ ]:
torch.cuda.empty_cache()
import gc

gc.collect()

1824

In [ ]:
# One forward pass
row = val_df_contexts_per_qe.iloc[0]
estimate_cmi(
    row["query_form"],
    entity=row["entity"],
    contexts=row["contexts"][:128],
    model=model,
    tokenizer=tokenizer,
    bs=BATCH_SZ,
)

Setting model.config.pad_token_id to model.config.eos_token_id


/home/kevin/code/rycolab/measureLM/measuring/estimate_probs.py:275: RuntimeWarning: invalid value encountered in divide
  return np.sum(prob_x_y_given_e * np.nan_to_num(np.log(prob_y_given_context_and_entity / prob_y_given_e)))


0.020289460580657594

In [ ]:
tqdm.pandas()
val_df_contexts_per_qe["susceptibility_score"] = val_df_contexts_per_qe.progress_apply(
    lambda row: estimate_cmi(
        query=row["query_form"],
        entity=row["entity"],
        contexts=row["contexts"],
        model=model,
        tokenizer=tokenizer,
        answer_map=None,
        bs=BATCH_SZ,
    ),
    axis=1,
)
val_df_contexts_per_qe.to_csv(val_results_path)

  0%|          | 0/10 [00:00<?, ?it/s]

/home/kevin/code/rycolab/measureLM/measuring/estimate_probs.py:275: RuntimeWarning: invalid value encountered in divide
  return np.sum(prob_x_y_given_e * np.nan_to_num(np.log(prob_y_given_context_and_entity / prob_y_given_e)))
 40%|████      | 4/10 [00:00<00:00, 32.33it/s]/home/kevin/code/rycolab/measureLM/measuring/estimate_probs.py:275: RuntimeWarning: divide by zero encountered in log
  return np.sum(prob_x_y_given_e * np.nan_to_num(np.log(prob_y_given_context_and_entity / prob_y_given_e)))
100%|██████████| 10/10 [00:00<00:00, 25.08it/s]


In [ ]:
# After loading/preprocessing your dataset, log it as an artifact to W&B
if LOG_DATASETS:
    print(f"Logging results to w&b run {wandb.run}.")
    artifact = wandb.Artifact(name=data_id, type="dataset")
    artifact.add_dir(local_path=data_dir)
    run.log_artifact(artifact)

wandb: Adding directory to artifact (./data/CountryCapital/CountryCapital-mc15-me5-cappertype/0)... Done. 0.0s


Logging results to w&b run <wandb.sdk.wandb_run.Run object at 0x7fd45e01fe20>.


### Evaluate Model

In [ ]:
val_df_contexts_per_qe["entity"].value_counts()

entity
(Zimbabwe,)          2
(The Sticklands,)    2
(Jakana,)            2
(Mordor,)            2
(Baekje,)            2
Name: count, dtype: int64

In [ ]:
val_df_contexts_per_qe[
    val_df_contexts_per_qe["query_form"] == "The capital of {} is"
].sort_values(by="susceptibility_score")

,q_id,query_form,entity,contexts,susceptibility_score
5,capital_of,The capital of {} is,"(Zimbabwe,)","[The capital of Baekje is Doha.\n, The capital...",0.400978
6,capital_of,The capital of {} is,"(The Sticklands,)","[The capital of Baekje is Doha.\n, The capital...",0.479788
8,capital_of,The capital of {} is,"(Mordor,)","[The capital of Baekje is Doha.\n, The capital...",0.568573
9,capital_of,The capital of {} is,"(Baekje,)","[The capital of Baekje is Doha.\n, The capital...",0.648642
7,capital_of,The capital of {} is,"(Jakana,)","[The capital of Baekje is Doha.\n, The capital...",0.651999


In [ ]:
val_df_contexts_per_qe[
    val_df_contexts_per_qe["query_form"] == "Q: What is the capital of {}?\nA:"
].sort_values(by="susceptibility_score")

,q_id,query_form,entity,contexts,susceptibility_score
0,capital_of,Q: What is the capital of {}?\nA:,"(Zimbabwe,)","[The capital of Baekje is Doha.\n, The capital...",0.020289
4,capital_of,Q: What is the capital of {}?\nA:,"(Baekje,)","[The capital of Baekje is Doha.\n, The capital...",0.036602
2,capital_of,Q: What is the capital of {}?\nA:,"(Jakana,)","[The capital of Baekje is Doha.\n, The capital...",0.047524
3,capital_of,Q: What is the capital of {}?\nA:,"(Mordor,)","[The capital of Baekje is Doha.\n, The capital...",0.059646
1,capital_of,Q: What is the capital of {}?\nA:,"(The Sticklands,)","[The capital of Baekje is Doha.\n, The capital...",0.068201


In [ ]:
wandb.finish()